<div class="alert alert-info">
<u><strong>Authors:</strong></u> <b>Alberto Vavassori</b> (alberto.vavassori@polimi.it), <b>Emanuele Capizzi</b> (emanuele.capizzi@mail.polimi.it), <b>Vasil Yordanov</b> (vasil.yordanov@polimi.it) - 2024 - Politecnico di Milano, Italy <br>
Developed within the LCZ-ODC project, funded by the Italian Space Agency (agreement n. 2022-30-HH.0).
</div>

# PRISMA pan-sharpening: quality assessment

This Notebook implements **quality assessment** of pansharpened PRISMA images ([Dhore and Veena 2015<sup>1</sup>](#1); [Helmy and El-Tawel 2015<sup>2</sup>](#2)). Quality indexes are implemented in the `metrics.py` file. Metrics computation is adopted from a dedicated GitHub Repository[<sup>3</sup>](#3).
Quality assessment is carried out following the guidelines of the Wald's protocol[<sup>4</sup>](#4).

### Resources
<span id="1">[<sup>1</sup>Dhore, A.D. and Veena, C.S. «Evaluation of various pansharpening methods using image quality metrics». In Proceedings of the 2nd International Conference on Electronics and Communication Systems (ICECS), Coimbatore, India, 2015, 871–877.](https://ieeexplore.ieee.org/document/7125039)</span><br>
<span id="2">[<sup>2</sup>Helmy, A.K. and El-Tawel, G.S. «An integrated scheme to improve pan-sharpening visual quality of satellite images». *Egyptian Informatics Journal* 2015, 16(1), 121–131. doi: 10.1016/j.eij.2015.02.003](https://www.sciencedirect.com/science/article/pii/S1110866515000079#:~:text=Formula%20of%20ERGAS%3A,N%20represents%20no%20of%20bands)</span><br>
<span id="3">[<sup>3</sup>GitHub Repo for pansharpening quality assessment](https://github.com/wasaCheney/IQA_pansharpening_python)</span><br>
<span id="4">[<sup>4</sup>Wald, L. et al. «Fusion of satellite images of different spatial resolutions: Assessing the quality of resulting images». *Photogrammetric engineering and remote sensing* 1997, 63(6), 691-699](https://hal.science/hal-00365304/)</span>

### <a id='TOC_TOP'></a>Notebook content

</div>
    
 1. [Libraries and Data Preparation](#sec1)
 2. [Visual inspection of the pansharpening quality](#sec2)
 3. [Spectral distorsions on the training samples](#sec3)
 4. [Quantitative assessment of the pansharpening quality: quality indexes computation](#sec4)

<hr>

<div class="alert alert-info" role="alert">
    
## <a id='sec1'></a>&#x27A4; 1. Libraries and Data Preparation

[Back to top](#TOC_TOP)
    
</div>

### Import useful libraries

In [ ]:
import numpy as np
import rasterio as rio
from rasterio.crs import CRS
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.merge import merge
from rasterio.plot import show_hist
from rasterio.warp import reproject, Resampling
import matplotlib.pyplot as plt
import matplotlib.image
import ipywidgets as widgets
from sklearn.decomposition import PCA
import geopandas as gpd
import h5py
import pandas as pd
import pyproj
import cv2
import json
import xml.dom.minidom
from shapely.geometry import Polygon
from rasterio import mask
from shapely.geometry import box
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, LayersControl, Rectangle
import leafmap

In [ ]:
# Import functions and set auto-reload
from methods import *
from metrics import *
from functions import *
%load_ext autoreload
%autoreload 2

### Date selection

Here it is possible to select the PRISMA image acquisition date:

In [ ]:
date_prisma_w = widgets.Dropdown(
    options = ['2023-02-09', '2023-03-22', '2023-04-08', '2023-06-17', '2023-07-10', '2023-08-08'],
    value = '2023-02-09',
    description = 'PRISMA date:',
    disabled = False,
    layout = {'width': 'max-content'},
    style = {'description_width': 'initial'}
)
date_prisma_w

In [ ]:
sel_prisma_date = date_prisma_w.value
print(f"The selected date is {sel_prisma_date}.")

Number of bands on which pansharpening is performed:

In [ ]:
n_bands = 63

According to the selected date, the folder containing the coregistered images (where the outputs of the Notebook will be saved as well) is the following:

In [ ]:
prisma_path = 'coregistered/' + sel_prisma_date.replace('-', '') + '/'
prisma_path

In [ ]:
sel_prisma_date = date_prisma_w.value
selected_prisma_image = prisma_path + 'hs_VNIR_1.tif'
selected_prisma_pan = prisma_path + 'pan_1.tif'
selected_prisma_image_5m = prisma_path + 'hs_5m_1_nn.tif'

print(f"The selected date is --> {sel_prisma_date}.")
print(f"The selected PRISMA image is --> {selected_prisma_image}.")
print(f"The selected PRISMA image (at 5m) is --> {selected_prisma_image_5m}.")

<div class="alert alert-info" role="alert">
    
## <a id='sec2'></a>&#x27A4; 2. Visual inspection of the pansharpening quality

[Back to top](#TOC_TOP)
    
</div>

In this part of the Notebook, it is possible to export the RGB pansharpened images and inspect visually the result quality.

In [ ]:
sw = widgets.RadioButtons(
    options=['Principal Component Analysis', 'Gram-Schmidt', 'Gram-Schmidt Adaptive'],
    description='Image to visualize',
    disabled=False,
    value='Gram-Schmidt Adaptive'
)
sw

In [ ]:
if sw.value == 'Principal Component Analysis':
    image_pansharpened_path = prisma_path + 'pansharpened_PCA_1.tif'
elif sw.value == 'Gram-Schmidt':
    image_pansharpened_path = prisma_path + 'pansharpened_GS_1.tif'
else: image_pansharpened_path = prisma_path + 'pansharpened_GSA_1.tif'

In [ ]:
selected_prisma_pan_image = image_pansharpened_path
selected_prisma_pan_image

<div class="alert alert-success" role="alert">
<span>&#x2714;</span>
<a id='libraries'></a>
First, select which of the three pansharpened images you want to export and visualize.
</div>

In [ ]:
# original PRISMA image (30m)
with rio.open(selected_prisma_image) as src_hs:
    hs_data = src_hs.read()
    hs_data_meta = src_hs.meta

In [ ]:
# panchromatic PRISMA band (5m)
with rio.open(selected_prisma_pan) as src_pan:
    pan_data = src_pan.read()
    pan_data_meta = src_pan.meta

In [ ]:
# PRISMA image interpolated at 5m (nearest neighbour)
with rio.open(selected_prisma_image_5m) as src:
    hs_5m_data = src.read()
    hs_5m_data_meta = src.meta

In [ ]:
# pansharpened PRISMA image
with rio.open(image_pansharpened_path) as src_p:
    image_pansharpened = src_p.read()
    pansharpened_meta = src_p.meta

In [ ]:
image_pansharpened.shape

In [ ]:
# Update the number of bands to 3 before running the function
dst_meta = pansharpened_meta
dst_meta['count'] = 3

The following function `convert_to_RGB` will create RGB images from the pansharpened and the original HS image. This is intended for both easier visualization in a GIS software and for convertion to PNG image.

In [ ]:
data_pansh, data_hs = convert_to_RGB(prisma_path, image_pansharpened, hs_5m_data, pansharpened_meta)

Export the created RGB image to JPG.

In [ ]:
if sw.value == 'Principal Component Analysis':
    matplotlib.image.imsave(prisma_path + 'validation/' + 'image_pansharpened_PCA.jpg', data_pansh, vmax = 0.5)
    matplotlib.image.imsave(prisma_path + 'validation/' + 'image_original.jpg', data_hs, vmax = 0.5)
    img_pansh = prisma_path + 'validation/' + 'image_pansharpened_PCA.jpg'
    image_orig = prisma_path + 'validation/' + 'image_original.jpg'
elif sw.value == 'Gram-Schmidt':
    matplotlib.image.imsave(prisma_path + 'validation/' + 'image_pansharpened_GS.jpg', data_pansh, vmax = 0.5)
    matplotlib.image.imsave(prisma_path + 'validation/' + 'image_original.jpg', data_hs, vmax = 0.5)
    img_pansh = prisma_path + 'validation/' + 'image_pansharpened_GS.jpg'
    image_orig = prisma_path + 'validation/' + 'image_original.jpg'
elif sw.value == 'Gram-Schmidt Adaptive':
    matplotlib.image.imsave(prisma_path + 'validation/' + 'image_pansharpened_GSA.jpg', data_pansh)#, vmax = 0.5
    matplotlib.image.imsave(prisma_path + 'validation/' + 'image_original.jpg', data_hs)#, vmax = 0.5
    img_pansh = prisma_path + 'validation/' + 'image_pansharpened_GSA.jpg'
    image_orig = prisma_path + 'validation/' + 'image_original.jpg'

Display an interactive visualization for easy comparison of the original and pansharpened images in the area of interest.

In [ ]:
leafmap.image_comparison(
    image_orig,
    img_pansh,
    label1='Original Image',
    label2='Pansharpened Image',
    starting_position=50,
    width=1000
)

<div class="alert alert-warning" role="alert">
<span>&#9888;</span>
<a id='warning'></a> If the interactive image does not appear, run again the last code block.
</div>

<div class="alert alert-info" role="alert">
    
## <a id='sec3'></a>&#x27A4; 3. Spectral distorsions on the training samples

[Back to top](#TOC_TOP)
    
</div>

In this part, the spectral signatures, before and after pansharpening, of the training samples used for LCZ classification are plotted.

Open the JSON files to retrieve the PRISMA and Sentinel-2 band central wavelengths:

In [ ]:
with open('./layers/PRISMA_wvl.json', "r") as json_file:
    wvl = json.load(json_file)

In [ ]:
with open('./layers/S2_wvl.json', "r") as json_file:
    wvl_s = json.load(json_file)

Import the geopackages containing pre-defined training samples and the boundary of the area of interest (i.e. the Metropolitan City of Milan):

In [ ]:
training_folder = './layers/training_samples/training_set_' + sel_prisma_date.replace('-', '') + '.gpkg'
cmm_folder = './layers/CMM.gpkg'

In [ ]:
legend = {
    2: ['Compact mid-rise', '#D10000'],
    3: ['Compact low-rise', '#CD0000'],
    5: ['Open mid-rise', '#FF6600'],
    6: ['Open low-rise', '#FF9955'],
    8: ['Large low-rise', '#BCBCBC'],
    101: ['Dense trees', '#006A00'],
    102: ['Scattered trees', '#00AA00'],
    104: ['Low plants', '#B9DB79'],
    105: ['Bare rock or paved', '#545454'],
    106: ['Bare soil or sand', '#FBF7AF'],
    107: ['Water', '#6A6AFF']
}

In [ ]:
training, m, shapes = plot_training_samples(training_folder, cmm_folder, legend)
m

Compute *median*, *mean*, and *standard deviation* of the spectral signatures of the training samples from **PRISMA**:

In [ ]:
spectral_sign_median, spectral_sign_std = compute_spectral_signature(selected_prisma_image, legend, shapes)

Compute *median*, *mean*, and *standard deviation* of the spectral signatures of the training samples from **pansharpened PRISMA**:

In [ ]:
spectral_sign_median_s, spectral_sign_std_s = compute_spectral_signature(selected_prisma_pan_image, legend, shapes)

Select the LCZ classes of interest. The median spectral signature, as well as the +/- standard deviation interval for the selected classes will be displayed in the following.

In [ ]:
LCZ_names = [value[0] for value in legend.values()]
checkboxes = [widgets.Checkbox(value=True, description=str(LCZ)) for LCZ in LCZ_names]
output = widgets.VBox(children=checkboxes)
output

In [ ]:
selected_LCZ_names = [checkbox.description for checkbox in checkboxes if checkbox.value]
selected_classes = [key for key, value in legend.items() if value[0] in selected_LCZ_names]

In [ ]:
plot_spectral_sign_comparison(wvl, spectral_sign_median, spectral_sign_median_s, legend, selected_classes)

<div class="alert alert-info" role="alert">
    
## <a id='sec4'></a>&#x27A4; 4. Quantitative assessment of the pansharpening quality: quality indexes computation

[Back to top](#TOC_TOP)
    
</div>

In this part some metrics are calculated to quantitatively assess the quality of the pansharpened images, following the **reduced resolution (RR)** approach, according to the *Wald's protocol*.
RR approach measures the similarity of the fused product to an ideal reference, namely the original HS image.
Accordingly, the resolutions of the original HS and PAN images are degradated and the fusion is performed on the degraded data.

The following quality measures are used:
1. *Spectral Angle Mapper (SAM)* that measures spectral quality;
2. *Erreur Relative Globale Adimensionnelle de Synthése (ERGAS)* which is a global adimensional quality index based on the RMSE;
3. *Spatial Correlation Coefficient (SCC)* that measure spatial quality;
4. *Peak Signal-to-Noise Ratio (PSNR)*.

The first step is to degrade the both the hyperspectral and the panchromatic bands by a factor of 6 (which is the ratio between the spatial resolution of the hyperspectral and panchromatic images). Accodingly, it is necessary to set the metadata of the degraded images.

In [ ]:
degrade_resolution_factor = 6

In [ ]:
# metadata of the original hyperspectral image (VNIR bands)
hs_data_meta

In [ ]:
# metadata of the pansharpened image
pansharpened_meta

Metadata of the degraded hyperspectral image (VNIR bands):

In [ ]:
hs_data_degraded_meta = hs_data_meta.copy()
hs_data_degraded_meta.update({
    'height': int(src_hs.height / degrade_resolution_factor),
    'width': int(src_hs.width / degrade_resolution_factor),
    'transform': rio.Affine(src_hs.transform[0] * degrade_resolution_factor, 0, src_hs.bounds.left, 
                            0, - (src_hs.transform[0] * degrade_resolution_factor), src_hs.bounds.top),
    'count': n_bands
})

In [ ]:
hs_data_degraded_meta #metadata of the downgraded hs image (30x6 m)

Metadata of the degraded panchromatic image:

In [ ]:
# set metadata of the degraded pan image
pan_data_degraded_meta = pansharpened_meta.copy()
pan_data_degraded_meta.update({
    'height': int(src_pan.height / degrade_resolution_factor),
    'width': int(src_pan.width / degrade_resolution_factor),
    'transform': rio.Affine(src_pan.transform[0] * degrade_resolution_factor, 0, src_pan.bounds.left, 
                            0, - (src_pan.transform[0] * degrade_resolution_factor), src_pan.bounds.top),
    'dtype' : 'float32',
    'count': 1
})

In [ ]:
pan_data_degraded_meta #metadata of the downgraded pan image (5x6 m)

Now it is possible to actually downscale the hyperspectral and panchromatic bands and save them to GeoTIFF files. First select the downsampling method.

In [ ]:
resampling_methods = { "nearest": Resampling.nearest,
                      "bilinear": Resampling.bilinear,
                      "cubic": Resampling.cubic,
                      "cubic_spline": Resampling.cubic_spline,
                      "lanczos": Resampling.lanczos,
                      "average": Resampling.average
}

resampling_methods_list = list(resampling_methods.keys())

resampling_w = widgets.Dropdown(
    options=resampling_methods_list,
    value='bilinear',
    description='Method:',
    disabled=False,
)
resampling_w

In [ ]:
degraded_hs_path = prisma_path + 'validation/' + 'hs_degraded_180m.tif'
hs_data_degraded = resample_image(hs_data, hs_data_meta, hs_data_degraded_meta, degraded_hs_path, resampling_methods[resampling_w.value])
print(f"Resampling of HS image using {resampling_w.value} done")

In [ ]:
degraded_pan_path = prisma_path + 'validation/' + 'pan_degraded_30m.tif'
pan_data_degraded = resample_image(pan_data, pan_data_meta, pan_data_degraded_meta, degraded_pan_path, resampling_methods[resampling_w.value])
print(f"Resampling of PAN image using {resampling_w.value} done")

Before applying again the pansharpening methods, the degraded hyperspectral image has to be resampled to match the number of pixels of the degraded panchromatic image (using the same interpolation method as above).

In [ ]:
dst_meta = pan_data_degraded_meta.copy()
dst_meta['count'] = n_bands
dst_meta

In [ ]:
degraded_hs_upsampled_path = prisma_path + 'validation/' + 'hs_degraded_upsampled_30m.tif'
hs_data_degraded_upsampled = resample_image(hs_data_degraded, hs_data_degraded_meta, dst_meta, degraded_hs_upsampled_path, resampling_methods[resampling_w.value])
print(f"Resampling of HS image using {resampling_w.value} done")

Apply again the same pansharpening methods to the two downgraded images.

In [ ]:
if sw.value == 'Principal Component Analysis':
    pansharpened, variance_ratios, pc_comp = pan_pca(pan_data_degraded, hs_data_degraded_upsampled)
    with rio.open(prisma_path + 'validation/' + 'pansharpened_PCA_validation.tif', 'w', **dst_meta) as dst:
        dst.write(pansharpened)
elif sw.value == 'Gram-Schmidt':
    pansharpened = pan_GS(pan_data_degraded, hs_data_degraded_upsampled)
    with rio.open(prisma_path + 'validation/' + 'pansharpened_GS_validation.tif', 'w', **dst_meta) as dst:
        dst.write(pansharpened)
elif sw.value == 'Gram-Schmidt Adaptive':
    pansharpened = pan_GSA(pan_data_degraded, hs_data_degraded, hs_data_degraded_upsampled, 'local')
    with rio.open(prisma_path + 'validation/' + 'pansharpened_GSA_validation.tif', 'w', **dst_meta) as dst:
        dst.write(pansharpened)

#### 1. Spectral Angle Mapper (SAM)

In [ ]:
print("The optimum value is 0.")
print(f"SAM applied on {sw.value} pansharpened image: {sam(pansharpened, hs_data):.3f}")

#### 2. Erreur Relative Globale Adimensionnelle de Synthése (ERGAS)

In [ ]:
print("The optimum value is 0.")
print(f"ERGAS applied on {sw.value} pansharpened image: {ergas(pansharpened, hs_data):.3f}")

#### 3. Spatial Correlation Coefficient (SCC)

In [ ]:
print("The optimum value is 1.")
print(f"SCC applied on {sw.value} pansharpened image: {scc(pansharpened, hs_data):.3f}")

#### 4. Peak Signal-to-Noise Ratio (PSNR)

In [ ]:
print("Higher PSNR is better.")
print(f"PSNR applied on {sw.value} pansharpened image: {qindex(pansharpened, hs_data):.3f}")

#### 5. Spectral Distorsion Index (D)

In [ ]:
print(f"D lambda applied on {sw.value} pansharpened image: {D_lambda(image_pansharpened, hs_data):.3f}")